In [1]:
import os
import sys
sys.path.append("..")

import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision

#from data import loader_train, loader_val, loader_test
from train import train_model, test_model
from models import getModels
from utils import show_img
from trainers import Regression, Classification, BinaryClassification
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from ResNet18 import ResNet, BasicBlock

In [2]:
from medmnist import PneumoniaMNIST, BreastMNIST, RetinaMNIST

from torch.utils.data import Subset

batch_size = 128
transfomrms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])
test_dataset = RetinaMNIST(split="test",transform= transfomrms, download=True,size=224)
train_dataset = RetinaMNIST(split="train", transform= transfomrms,download=True,size=224)
val_dataset = RetinaMNIST(split="val",transform=transfomrms, download=True,size=224)

loader_train = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
loader_val = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
loader_test = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
# Definir el tamaño del subconjunto que deseas usar (por ejemplo, 10%)
subset_size = int(0.1 * len(train_dataset))

# Crear subconjuntos con índices aleatorios
train_indices = torch.randperm(len(train_dataset))[:subset_size]
val_indices = torch.randperm(len(val_dataset))[:subset_size*2]
test_indices = torch.randperm(len(test_dataset))[:subset_size*2]

# Crear los subconjuntos utilizando los índices seleccionados
train_subset = Subset(train_dataset, train_indices)
val_subset = Subset(val_dataset, val_indices)
test_subset = Subset(test_dataset, test_indices)

Using downloaded and verified file: C:\Users\34658\.medmnist\retinamnist_224.npz
Using downloaded and verified file: C:\Users\34658\.medmnist\retinamnist_224.npz
Using downloaded and verified file: C:\Users\34658\.medmnist\retinamnist_224.npz


In [3]:
# start a new wandb run to track this script
import wandb
wandb.finish()
wandb.init(
    # set the wandb project where this run will be logged
    project="Test",
    name="MyComputer - Tests",

    # track hyperparameters and run metadata
    config={
        "N_images" : 1,
        "learning_rate": 0.001,
        "dataset": "Expert 1",
        "epochs": 50,
    }
)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chermar (chermar-universitat-polit-cnica-de-val-ncia). Use `wandb login --relogin` to force relogin


In [4]:
import os
import sys
sys.path.append("..")


loader_train = DataLoader(train_subset, batch_size=32, shuffle=True)
loader_val = DataLoader(val_dataset, batch_size=32, shuffle=True)
loader_test = DataLoader(test_subset, batch_size=32, shuffle=True)

In [5]:
batch = next(iter(loader_train))
_, y = batch
print(y[0])
model =  ResNet(img_channels=3, num_layers=18, block=BasicBlock, num_classes=5)

y_hat = model(batch)


tensor([0], dtype=torch.int32)


TypeError: conv2d() received an invalid combination of arguments - got (list, Parameter, NoneType, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!list of [Tensor, Tensor]!, !Parameter!, !NoneType!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!list of [Tensor, Tensor]!, !Parameter!, !NoneType!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, int)


In [ ]:
#train a model
model =  ResNet(img_channels=3, num_layers=18, block=BasicBlock, num_classes=1)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
trainer = Regression(model, device, num_classes = 5)


train_model(model, loader_train, loader_val, trainer,  device, num_epochs=2, classification = False)
test_model(model, loader_test, trainer, device, classification = True)